In [1]:
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
import re
from tqdm import tqdm
from datetime import datetime
plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
data = pd.read_csv("Data/Preprocessed/순환분석.csv")

In [4]:
data.isna().sum()

사용자코드             0
연도                0
분기                0
날짜                0
요일                0
시간대명              0
성별                0
연령대               0
사용자 사업자 번호     3049
사용자 업종         3049
사용자 시군 대분류        0
사용자 시도            0
사용자 시군            0
사용자 읍면            0
가맹점 시군 대분류        0
가맹점 시도            0
가맹점 시군            0
가맹점 사업자 번호        0
가맹점 업종            0
가맹점 읍면            0
사용금액              0
사용자 가맹점명      48678
사용자 골목상권분류    48678
사용자 대분류       48678
사용자 중분류       48678
사용자 소분류       48678
사용자 가맹점 주소    48678
사용자 가맹가입일자    48678
가맹점 가맹점명          1
가맹점 골목상권분류        1
가맹점 대분류           1
가맹점 중분류           1
가맹점 소분류           1
가맹점 가맹점 주소        1
가맹점 가맹가입일자        1
dtype: int64

In [5]:
#삭제된가맹점 '폐업'으로 바꾸고, 그 row의 NA값들을 '페업'으로 채우기 
data.loc[data['사용자 사업자 번호'] == '삭제된가맹점', '사용자 사업자 번호'] = '폐업'
data.loc[data['사용자 사업자 번호'] == '폐업', :] = data.loc[data['사용자 사업자 번호'] == '폐업', :].fillna('폐업')
#결측치 모두 제거하기 
data = data.dropna()
data.isna().sum()

사용자코드         0
연도            0
분기            0
날짜            0
요일            0
시간대명          0
성별            0
연령대           0
사용자 사업자 번호    0
사용자 업종        0
사용자 시군 대분류    0
사용자 시도        0
사용자 시군        0
사용자 읍면        0
가맹점 시군 대분류    0
가맹점 시도        0
가맹점 시군        0
가맹점 사업자 번호    0
가맹점 업종        0
가맹점 읍면        0
사용금액          0
사용자 가맹점명      0
사용자 골목상권분류    0
사용자 대분류       0
사용자 중분류       0
사용자 소분류       0
사용자 가맹점 주소    0
사용자 가맹가입일자    0
가맹점 가맹점명      0
가맹점 골목상권분류    0
가맹점 대분류       0
가맹점 중분류       0
가맹점 소분류       0
가맹점 가맹점 주소    0
가맹점 가맹가입일자    0
dtype: int64

In [6]:
#골목상권=='기타'에서 대분류가 '기타'인 것들 제거 
data[(data['가맹점 골목상권분류'] == '기타') & (data['가맹점 대분류'] == '기타')]
##확인해보니, 그런 데이터들 없어서 패스 

,사용자코드,연도,분기,날짜,요일,시간대명,성별,연령대,사용자 사업자 번호,사용자 업종,...,사용자 소분류,사용자 가맹점 주소,사용자 가맹가입일자,가맹점 가맹점명,가맹점 골목상권분류,가맹점 대분류,가맹점 중분류,가맹점 소분류,가맹점 가맹점 주소,가맹점 가맹가입일자


In [7]:
#부여군에서 [부여읍, 규암면, 홍산면, 기타]로 분류 
print(data.loc[(data['사용자 시군 대분류'] == '부여군') & (data['가맹점 시군 대분류'] == '부여군')]['사용자 읍면'].unique())
print(data.loc[(data['사용자 시군 대분류'] == '부여군') & (data['가맹점 시군 대분류'] == '부여군')]['가맹점 읍면'].unique())

['부여읍' '구룡면' '규암면' '장암면' '임천면' '은산면' '석성면' '홍산면' '외산면' '내산면' '옥산면' '세도면'
 '초촌면' '양화면' '남면' '충화면']
['부여읍' '규암면' '임천면' '은산면' '홍산면' '석성면' '외산면' '구룡면' '장암면' '남면' '세도면' '초촌면'
 '양화면' '옥산면' '내산면' '충화면']


In [8]:
#지역 4개로 나누기 
def location(data):
    values = ['부여읍', '규암면', '홍산면']
    condition1 = [
        (data['사용자 읍면'] == '부여읍'),
        (data['사용자 읍면'] == '규암면'),
        (data['사용자 읍면'] == '홍산면')
    ] 
    condition2 = [
        (data['가맹점 읍면'] == '부여읍'),
        (data['가맹점 읍면'] == '규암면'),
        (data['가맹점 읍면'] == '홍산면')
    ]
    data.loc[:, '사용자 읍면'] = np.select(condition1, values, default='기타')
    data.loc[:, '가맹점 읍면'] = np.select(condition2, values, default='기타')
    return data

buyeo = data.loc[(data['사용자 시군 대분류'] == '부여군') & (data['가맹점 시군 대분류'] == '부여군')]      #len(buyeo) = 437290 (기존: 444233)
data = location(buyeo)
print(data['사용자 읍면'].unique())
print(data['가맹점 읍면'].unique())

['부여읍' '기타' '규암면' '홍산면']
['부여읍' '규암면' '기타' '홍산면']


In [9]:
#날짜 형태로 변환 
def convert_date(data):
    data.loc[:, '연도'] = data['연도'].str.replace('년', '')
    data['날짜'] = pd.to_datetime(data['날짜'], format='%Y-%m-%d')
    data['연월'] = data['날짜'].dt.strftime('%Y-%m')
    return data

data = convert_date(data)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 437290 entries, 0 to 444232
Data columns (total 36 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   사용자코드       437290 non-null  object        
 1   연도          437290 non-null  object        
 2   분기          437290 non-null  int64         
 3   날짜          437290 non-null  datetime64[ns]
 4   요일          437290 non-null  object        
 5   시간대명        437290 non-null  object        
 6   성별          437290 non-null  object        
 7   연령대         437290 non-null  object        
 8   사용자 사업자 번호  437290 non-null  object        
 9   사용자 업종      437290 non-null  object        
 10  사용자 시군 대분류  437290 non-null  object        
 11  사용자 시도      437290 non-null  object        
 12  사용자 시군      437290 non-null  object        
 13  사용자 읍면      437290 non-null  object        
 14  가맹점 시군 대분류  437290 non-null  object        
 15  가맹점 시도      437290 non-null  object        
 16  가맹점 시군 

In [11]:
#연도별 데이터프레임 나누기 
date_ranges = [('2020-11-01', '2021-10-31'), ('2021-11-01', '2022-10-31'), ('2022-11-01', '2023-10-31')]
year_dict = {}
for idx, (start_date, end_date) in enumerate(date_ranges):
    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)
    data_filt = data[(data['날짜'] >= start_date_dt) & (data['날짜'] <= end_date_dt)]
    year_dict[f"df_{idx}"] = data_filt

for key, value in year_dict.items():
    globals()[f'{key}'] = value 
##df_0, df_1, df_2 

#### 순환업종 집계

In [12]:
def eda(data, group_by_columns):
    # 총 사용금액
    a = data.groupby(group_by_columns)['사용금액'].sum().reset_index()
    a = a.rename(columns={'사용금액': '총 사용금액'})
    # 사용횟수
    b = data.groupby(group_by_columns)['사용금액'].count().reset_index()
    b = b.rename(columns={'사용금액': '사용횟수'})
    # 사용자 가맹점 수
    c = data.groupby(group_by_columns)['사용자 가맹점명'].nunique().reset_index()
    c = c.rename(columns={'사용자 가맹점명': '사용자 가맹점 수'})
    # Merge DataFrames
    merged_df = a.merge(b, on=group_by_columns, how='left').merge(c, on=group_by_columns, how='left')
    merged_df['1회당 평균 사용금액'] = round((merged_df['총 사용금액'] / merged_df['사용횟수']), 2)
    return merged_df

In [13]:
result0 = eda(df_0, ['가맹점 읍면', '가맹점 대분류', '가맹점 중분류', '가맹점 소분류', '사용자 읍면', '사용자 대분류', '사용자 중분류', '사용자 소분류'])
result1 = eda(df_1, ['가맹점 읍면', '가맹점 대분류', '가맹점 중분류', '가맹점 소분류', '사용자 읍면', '사용자 대분류', '사용자 중분류', '사용자 소분류'])
result2 = eda(df_2, ['가맹점 읍면', '가맹점 대분류', '가맹점 중분류', '가맹점 소분류', '사용자 읍면', '사용자 대분류', '사용자 중분류', '사용자 소분류'])

In [14]:
dataframes = [result0, result1, result2]
for idx, df in enumerate(dataframes):
    df_head = df.head()
    print(f"Df {idx}:")
    print(df_head)

Df 0:
  가맹점 읍면   가맹점 대분류   가맹점 중분류    가맹점 소분류 사용자 읍면 사용자 대분류  사용자 중분류    사용자 소분류  \
0    규암면  관광/여가/오락  무도/유흥/가무   룸살롱/단란주점    규암면   생활서비스  이/미용/건강      여성미용실   
1    규암면  관광/여가/오락        숙박  모텔/여관/여인숙    부여읍      음식       한식  한식/백반/한정식   
2    규암면      농업관련       농자재        농자재    규암면    농업관련      농자재        농자재   
3    규암면      농업관련       농자재        농자재    규암면      소매  음/식료품소매      수산물소매   
4    규암면      농업관련       농자재        농자재    규암면     자동차  주유소/충전소        주유소   

   총 사용금액  사용횟수  사용자 가맹점 수  1회당 평균 사용금액  
0   50000     1          1     50000.00  
1   45000     1          1     45000.00  
2  169960     3          3     56653.33  
3   11500     1          1     11500.00  
4   24000     1          1     24000.00  
Df 1:
  가맹점 읍면   가맹점 대분류   가맹점 중분류   가맹점 소분류 사용자 읍면 사용자 대분류  사용자 중분류      사용자 소분류  \
0    규암면  관광/여가/오락  무도/유흥/가무  룸살롱/단란주점    부여읍   생활서비스  이/미용/건강        여성미용실   
1    규암면      농업관련       농자재       농자재    규암면    농업관련      농자재          농자재   
2    규암면      농업관련       농자재  

In [15]:
for idx, df in enumerate(dataframes):
    na_sum = df.isna().sum()
    print(f"Df {idx}:")
    print(na_sum)

Df 0:
가맹점 읍면         0
가맹점 대분류        0
가맹점 중분류        0
가맹점 소분류        0
사용자 읍면         0
사용자 대분류        0
사용자 중분류        0
사용자 소분류        0
총 사용금액         0
사용횟수           0
사용자 가맹점 수      0
1회당 평균 사용금액    0
dtype: int64
Df 1:
가맹점 읍면         0
가맹점 대분류        0
가맹점 중분류        0
가맹점 소분류        0
사용자 읍면         0
사용자 대분류        0
사용자 중분류        0
사용자 소분류        0
총 사용금액         0
사용횟수           0
사용자 가맹점 수      0
1회당 평균 사용금액    0
dtype: int64
Df 2:
가맹점 읍면         0
가맹점 대분류        0
가맹점 중분류        0
가맹점 소분류        0
사용자 읍면         0
사용자 대분류        0
사용자 중분류        0
사용자 소분류        0
총 사용금액         0
사용횟수           0
사용자 가맹점 수      0
1회당 평균 사용금액    0
dtype: int64


In [16]:
for idx, df in enumerate(dataframes):
    describe_df = df.describe()
    print(f"Df {idx}:")
    print(describe_df)

Df 0:
             총 사용금액         사용횟수    사용자 가맹점 수   1회당 평균 사용금액
count  7.031000e+03  7031.000000  7031.000000  7.031000e+03
mean   7.854521e+05    18.427109     2.079505  4.375125e+04
std    7.152089e+06   132.099715     4.855531  1.427116e+05
min    2.000000e+01     1.000000     1.000000  2.000000e+01
25%    2.600000e+04     1.000000     1.000000  1.304377e+04
50%    7.440000e+04     3.000000     1.000000  2.350000e+04
75%    2.687900e+05     9.000000     2.000000  4.251750e+04
max    3.249705e+08  8155.000000   143.000000  5.483590e+06
Df 1:
             총 사용금액          사용횟수    사용자 가맹점 수   1회당 평균 사용금액
count  7.652000e+03   7652.000000  7652.000000  7.652000e+03
mean   9.057658e+05     20.868400     2.214846  4.368480e+04
std    7.653501e+06    157.728281     5.515046  1.243359e+05
min    1.000000e+01      1.000000     1.000000  1.000000e+01
25%    2.640000e+04      1.000000     1.000000  1.300000e+04
50%    8.000000e+04      3.000000     1.000000  2.366238e+04
75%    3.011700e+05  

In [17]:
result0['기간'] = '2020-11-01 ~ 2021.10-31'
result1['기간'] = '2021-11-01 ~ 2022.10-31'
result2['기간'] = '2022-11-01 ~ 2023.10-31'

result_merge = pd.concat([result0, result1, result2], axis=0)

In [18]:
result_merge = result_merge[['기간', '가맹점 읍면', '가맹점 대분류', '가맹점 중분류', '가맹점 소분류', '사용자 읍면', '사용자 대분류', '사용자 중분류', 
                             '사용자 소분류', '총 사용금액', '사용횟수', '사용자 가맹점 수', '1회당 평균 사용금액']]
result_merge = result_merge.rename(columns={'사용자 읍면': '사용자 가맹점 읍면', '사용자 대분류': '사용자 가맹점 대분류', '사용자 중분류': '사용자 가맹점 중분류', 
                                            '사용자 소분류': '사용자 가맹점 소분류', '가맹점 읍면': '사용처 가맹점 읍면', '가맹점 대분류': '사용처 가맹점 대분류', 
                                            '가맹점 중분류': '사용처 가맹점 중분류', '가맹점 소분류': '사용처 가맹점 소분류'})
result_merge

,기간,사용처 가맹점 읍면,사용처 가맹점 대분류,사용처 가맹점 중분류,사용처 가맹점 소분류,사용자 가맹점 읍면,사용자 가맹점 대분류,사용자 가맹점 중분류,사용자 가맹점 소분류,총 사용금액,사용횟수,사용자 가맹점 수,1회당 평균 사용금액
0,2020-11-01 ~ 2021.10-31,규암면,관광/여가/오락,무도/유흥/가무,룸살롱/단란주점,규암면,생활서비스,이/미용/건강,여성미용실,50000,1,1,50000.00
1,2020-11-01 ~ 2021.10-31,규암면,관광/여가/오락,숙박,모텔/여관/여인숙,부여읍,음식,한식,한식/백반/한정식,45000,1,1,45000.00
2,2020-11-01 ~ 2021.10-31,규암면,농업관련,농자재,농자재,규암면,농업관련,농자재,농자재,169960,3,3,56653.33
3,2020-11-01 ~ 2021.10-31,규암면,농업관련,농자재,농자재,규암면,소매,음/식료품소매,수산물소매,11500,1,1,11500.00
4,2020-11-01 ~ 2021.10-31,규암면,농업관련,농자재,농자재,규암면,자동차,주유소/충전소,주유소,24000,1,1,24000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377,2022-11-01 ~ 2023.10-31,홍산면,자동차,주유소/충전소,충전소,홍산면,소매,종합소매점,편의점,156010,5,2,31202.00
7378,2022-11-01 ~ 2023.10-31,홍산면,자동차,주유소/충전소,충전소,홍산면,음식,닭/오리요리,후라이드/양념치킨,269315,15,1,17954.33
7379,2022-11-01 ~ 2023.10-31,홍산면,자동차,주유소/충전소,충전소,홍산면,음식,분식,라면김밥분식,52005,1,1,52005.00
7380,2022-11-01 ~ 2023.10-31,홍산면,자동차,주유소/충전소,충전소,홍산면,음식,한식,한식/백반/한정식,311945,8,2,38993.12


In [19]:
result_merge.to_csv("Result/0300. 가맹점_EDA(연계성 및 프로모션)/가맹점순환집계.csv", index=False, encoding="cp949")

##### old code

In [76]:
##EDA를 위한 '사용자 대분류' + '가맹점 대분류' 정보 추가하기 
cat_info = data[['순환업종', '사용자 대분류', '가맹점 대분류']]
cat_info = cat_info.drop_duplicates()
cat_info

,순환업종,사용자 대분류,가맹점 대분류
0,폐업_//_음식,폐업,음식
1,음식_//_음식,음식,음식
2,스포츠_//_소매,스포츠,소매
3,소매_//_소매,소매,소매
5,병원_//_대형마트,병원,대형마트
...,...,...,...
214206,스포츠_//_관광/여가/오락,스포츠,관광/여가/오락
275749,대형마트_//_종교,대형마트,종교
322973,학문/교육_//_종교,학문/교육,종교
351139,스포츠_//_종교,스포츠,종교


In [77]:
result_df = pd.merge(result, cat_info, on='순환업종', how='left')
result_df

,가맹점 읍면 new,연도,월,순환업종,총 사용금액,사용금액 비율,사용빈도,사용자 수,1회당 평균 사용금액,사용자 대분류,가맹점 대분류
0,규암면,2020,11,관광/여가/오락_//_대형마트,40000,0.03,2,2,20000.0,관광/여가/오락,대형마트
1,규암면,2020,11,관광/여가/오락_//_소매,9000,0.01,1,1,9000.0,관광/여가/오락,소매
2,규암면,2020,11,관광/여가/오락_//_음식,17400,0.01,2,2,8700.0,관광/여가/오락,음식
3,규암면,2020,11,농업관련_//_농업관련,120600,0.10,1,1,120600.0,농업관련,농업관련
4,규암면,2020,11,농업관련_//_대형마트,80080,0.07,5,5,16016.0,농업관련,대형마트
...,...,...,...,...,...,...,...,...,...,...,...
8781,홍산면,2023,10,폐업_//_관광/여가/오락,188000,0.09,1,1,188000.0,폐업,관광/여가/오락
8782,홍산면,2023,10,폐업_//_병원,294050,0.14,8,8,36756.0,폐업,병원
8783,홍산면,2023,10,폐업_//_생활서비스,150000,0.07,1,1,150000.0,폐업,생활서비스
8784,홍산면,2023,10,폐업_//_소매,2089400,0.99,10,7,208940.0,폐업,소매


In [80]:
#add '연-월' 칼럼 
result_df['연도'] = result_df['연도'].astype(str)
result_df['월'] = result_df['월'].astype(str)
result_df['연월'] = result_df['연도'] + "-" + result_df['월']
result_df

,가맹점 읍면 new,연도,월,순환업종,총 사용금액,사용금액 비율,사용빈도,사용자 수,1회당 평균 사용금액,사용자 대분류,가맹점 대분류,연월
0,규암면,2020,11,관광/여가/오락_//_대형마트,40000,0.03,2,2,20000.0,관광/여가/오락,대형마트,2020-11
1,규암면,2020,11,관광/여가/오락_//_소매,9000,0.01,1,1,9000.0,관광/여가/오락,소매,2020-11
2,규암면,2020,11,관광/여가/오락_//_음식,17400,0.01,2,2,8700.0,관광/여가/오락,음식,2020-11
3,규암면,2020,11,농업관련_//_농업관련,120600,0.10,1,1,120600.0,농업관련,농업관련,2020-11
4,규암면,2020,11,농업관련_//_대형마트,80080,0.07,5,5,16016.0,농업관련,대형마트,2020-11
...,...,...,...,...,...,...,...,...,...,...,...,...
8781,홍산면,2023,10,폐업_//_관광/여가/오락,188000,0.09,1,1,188000.0,폐업,관광/여가/오락,2023-10
8782,홍산면,2023,10,폐업_//_병원,294050,0.14,8,8,36756.0,폐업,병원,2023-10
8783,홍산면,2023,10,폐업_//_생활서비스,150000,0.07,1,1,150000.0,폐업,생활서비스,2023-10
8784,홍산면,2023,10,폐업_//_소매,2089400,0.99,10,7,208940.0,폐업,소매,2023-10


In [81]:
result_df.to_csv("Result/0300. 가맹점_EDA(연계성 및 프로모션)/가맹점순환집계.csv", index=False, encoding="cp949")

#### 1년만 보기 (2022.11 ~ 2023.10)

In [17]:
data = pd.read_csv("Result/0300. 가맹점_EDA(연계성 및 프로모션)/가맹점순환집계.csv", encoding="cp949")

In [18]:
data.head()

,가맹점 읍면 new,연도,월,순환업종,총 사용금액,사용금액 비율,사용빈도,사용자 수,1회당 평균 사용금액,사용자 대분류,가맹점 대분류,연월
0,규암면,2020,11,관광/여가/오락_//_대형마트,40000,0.03,2,2,20000.0,관광/여가/오락,대형마트,2020-11
1,규암면,2020,11,관광/여가/오락_//_소매,9000,0.01,1,1,9000.0,관광/여가/오락,소매,2020-11
2,규암면,2020,11,관광/여가/오락_//_음식,17400,0.01,2,2,8700.0,관광/여가/오락,음식,2020-11
3,규암면,2020,11,농업관련_//_농업관련,120600,0.10,1,1,120600.0,농업관련,농업관련,2020-11
4,규암면,2020,11,농업관련_//_대형마트,80080,0.07,5,5,16016.0,농업관련,대형마트,2020-11


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8786 entries, 0 to 8785
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   가맹점 읍면 new   8786 non-null   object 
 1   연도           8786 non-null   int64  
 2   월            8786 non-null   int64  
 3   순환업종         8786 non-null   object 
 4   총 사용금액       8786 non-null   int64  
 5   사용금액 비율      8786 non-null   float64
 6   사용빈도         8786 non-null   int64  
 7   사용자 수        8786 non-null   int64  
 8   1회당 평균 사용금액  8786 non-null   float64
 9   사용자 대분류      8786 non-null   object 
 10  가맹점 대분류      8786 non-null   object 
 11  연월           8786 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 823.8+ KB


In [20]:
data['연월'] = pd.to_datetime(data['연월'], format='%Y-%m')

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8786 entries, 0 to 8785
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   가맹점 읍면 new   8786 non-null   object        
 1   연도           8786 non-null   int64         
 2   월            8786 non-null   int64         
 3   순환업종         8786 non-null   object        
 4   총 사용금액       8786 non-null   int64         
 5   사용금액 비율      8786 non-null   float64       
 6   사용빈도         8786 non-null   int64         
 7   사용자 수        8786 non-null   int64         
 8   1회당 평균 사용금액  8786 non-null   float64       
 9   사용자 대분류      8786 non-null   object        
 10  가맹점 대분류      8786 non-null   object        
 11  연월           8786 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 823.8+ KB


In [22]:
data_new = data.loc[data['연월'] >= '2022-11']

In [23]:
data_new = data_new.loc[data_new['사용자 대분류'] != '폐업']

In [25]:
data['사용자 대분류'].unique()

array(['관광/여가/오락', '농업관련', '대형마트', '병원', '생활서비스', '소매', '스포츠', '음식',
       '자동차', '종교', '폐업', '학문/교육', '부동산'], dtype=object)

In [24]:
data_new['사용자 대분류'].unique()

array(['관광/여가/오락', '농업관련', '대형마트', '병원', '생활서비스', '소매', '스포츠', '음식',
       '자동차', '종교', '학문/교육'], dtype=object)

In [16]:
data_new.to_csv("Result/0300. 가맹점_EDA(연계성 및 프로모션)/가맹점순환집계(1년).csv", index=False, encoding="cp949")